### **FAKE NEWS PREDICTION**
Fake news prediction involves the use of machine learning algorithms and natural language processing techniques to analyze content and predict the likelihood that a given piece of information is fake or real. By examining various features such as language patterns, source credibility, and social media propagation, these models aim to identify potential misinformation before it spreads widely, helping users make more informed decisions about the information they encounter online.

#### **About dataset :**


1.   id: unique id for dataset
2.   title: the title of news article
3.   text: content of article
4.   label: a label that marks whether the news article is real or fake








In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing the libraries

In [47]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Loading the dataset

In [3]:
data=pd.read_csv("/content/drive/MyDrive/datasets/fake_news.csv")

In [4]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
data.shape

(6335, 4)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [7]:
data.describe()

,Unnamed: 0
count,6335.000000
mean,5280.415627
std,3038.503953
min,2.000000
25%,2674.500000
50%,5271.000000
75%,7901.000000
max,10557.000000


In [8]:
#checking for null values
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [9]:
#label encoding
data['label'] = data['label'].replace({'REAL': 0, 'FAKE': 1})

In [10]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
#renaming a column
data.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [14]:
data['id'].nunique()

6335

In [15]:
#separating data and label
X=data.drop(columns='label',axis=1)
Y=data['label']

In [16]:
print(X)
print(Y)

         id                                              title  \
0      8476                       You Can Smell Hillary’s Fear   
1     10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2      3608        Kerry to go to Paris in gesture of sympathy   
3     10142  Bernie supporters on Twitter erupt in anger ag...   
4       875   The Battle of New York: Why This Primary Matters   
...     ...                                                ...   
6330   4490  State Department says it can't find emails fro...   
6331   8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332   8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333   4021  In Ethiopia, Obama seeks progress on peace, se...   
6334   4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  
0     Daniel Greenfield, a Shillman Journalism Fello...  
1     Google Pinterest Digg Linkedin Reddit Stumbleu...  
2     U.S. Secretary of State Joh

### **STEMMING**
Stemming is the process of reducing a word to its root word


In [17]:
port_stem=PorterStemmer()

In [18]:
def stemming(text):
  stemmed_text=re.sub('[^a-zA-Z]',' ',text)
  stemmed_text=stemmed_text.lower()
  stemmed_text=stemmed_text.split()
  stemmed_text=[port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text


In [19]:
data['text']=data['text'].apply(stemming)

In [20]:
print(data['text'])

0       daniel greenfield shillman journal fellow free...
1       googl pinterest digg linkedin reddit stumbleup...
2       u secretari state john f kerri said monday sto...
3       kayde king kaydeek novemb lesson tonight dem l...
4       primari day new york front runner hillari clin...
                              ...                        
6330    state depart told republican nation committe c...
6331    p pb stand plutocrat pentagon post oct wikimed...
6332    anti trump protest tool oligarchi reform alway...
6333    addi ababa ethiopia presid obama conven meet l...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: text, Length: 6335, dtype: object


In [21]:
data.head(5)

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,daniel greenfield shillman journal fellow free...,1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,googl pinterest digg linkedin reddit stumbleup...,1
2,3608,Kerry to go to Paris in gesture of sympathy,u secretari state john f kerri said monday sto...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,kayde king kaydeek novemb lesson tonight dem l...,1
4,875,The Battle of New York: Why This Primary Matters,primari day new york front runner hillari clin...,0


In [22]:
#separating data and label
X=data['text'].values
Y=data['label'].values
print(X)


['daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim scandal realli attack 

In [23]:
#converting textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)
X=vectorizer.transform(X)

In [24]:
print(X)

  (0, 43266)	0.0305089294798837
  (0, 43143)	0.018330936254574128
  (0, 42836)	0.03723430652002139
  (0, 42832)	0.024123281735309754
  (0, 42805)	0.03342157965058952
  (0, 42772)	0.02373158199733929
  (0, 42770)	0.008769006159949829
  (0, 42704)	0.01586413299776395
  (0, 42645)	0.016735394976427433
  (0, 42558)	0.028489938333701406
  (0, 42557)	0.01687960148746075
  (0, 42520)	0.0383844928542425
  (0, 42478)	0.04282388601939193
  (0, 42381)	0.034991614640544406
  (0, 42326)	0.03598071033413201
  (0, 42312)	0.01846435959961643
  (0, 42233)	0.013971188044141832
  (0, 42202)	0.06213972123547004
  (0, 42097)	0.011535614274685978
  (0, 42066)	0.027174758879218847
  (0, 41990)	0.028859093884229654
  (0, 41929)	0.032891625478617195
  (0, 41881)	0.01652269636508778
  (0, 41833)	0.053741305697756876
  (0, 41801)	0.07118481130240924
  :	:
  (6334, 3905)	0.02940423648335081
  (6334, 3904)	0.027151902202961768
  (6334, 3727)	0.03642146927330242
  (6334, 3720)	0.02107984191394252
  (6334, 3444)	0.0

**Spilitting the dataset to training & test data**

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

**Training the Model: Logistic Regression**

In [29]:
model = LogisticRegression()

In [30]:
model.fit(X_train, Y_train)

LogisticRegression()

**Evaluation:**
Accuracy Score

In [31]:
#accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [32]:
print("Accuracy score of training data: ",training_data_accuracy)

Accuracy score of training data:  0.9481057616416733


In [33]:
#accuracy score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [34]:
print("Accuracy score of test data: ",test_data_accuracy)

Accuracy score of test data:  0.914759273875296


**Making Predictive System**

In [46]:
X_new=X_test[19]

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]==[0]):
  print("The new is Real")
else:
  print("The news is fake")

[1]
The news is fake


In [44]:
print(Y_test[19])

1
